In [ ]:
using Gmsh: gmsh
using GridapGmsh
using Gridap
using Gridap.Geometry
using Gridap.TensorValues
using Plots
using SymPy

In [ ]:
const T0 = 300
const TAppMax = T0 + 50

const phiMaxApp = 1000.0
const phiMin = 0.0

const delt = 1e-2
const tMax = 1
const uMax = 2.5e-6
const AppVel  = uMax/tMax
const uMin = 0

uTran_Temp = 0.2*uMax
uTran2_EP = 0.05*uMax

In [ ]:
using SymPy
x ,x1 = symbols( "x ,x1 " , real = true )
heaviside(x) = 0.5 * (sign(x) + 1)
interval(x , a , b ) = heaviside(x - a ) - heaviside(x - b )
hS = uMax /10
F_Temp( x ) = ( T0 - TAppMax ) * interval(x , -4* hS + uMin , uTran_Temp )
wh(x ,x1 ) = (1/( sqrt(2* pi ) * hS ) ) * exp( -(x -x1 ) ^2/(2* hS ^2) )
smoothT = SymPy.integrate( F_Temp(x1 ) *wh(x ,x1 ) ,(x1 , -4* hS , uMax ) ) + TAppMax
plot(smoothT,0,uMax)

In [ ]:
using SymPy
x ,x1 = symbols( "x ,x1 " , real = true )
heaviside( x ) = 0.5 * ( sign( x ) + 1)
interval(x , a , b ) = heaviside(x - a ) - heaviside(x - b )
hS_pf = uMax /100
F_φ( x ) = ( phiMin - phiMaxApp ) * interval(x , -4* hS_pf + uMin , uTran2_EP )
wh(x ,x1 ) = (1/( sqrt(2* pi ) * hS_pf ) ) * exp( -(x -x1 ) ^2/(2* hS_pf ^2) )
smoothφ = SymPy.integrate( F_φ(x1)*wh(x, x1), (x1 , -4* hS_pf , uMax ) ) + phiMaxApp
plot( smoothφ ,0 , uMax )

In [ ]:
function Tfun( u )
    if u <= uTran_Temp
        return (( TAppMax - T0 ) / uTran_Temp ) * u + T0
    else
        return TAppMax
    end
end
plot( Tfun ,0 , uMax )

In [ ]:
function φ_fun(φ)
    if φ <= uTran2_EP
        return ((phiMaxApp - phiMin ) / uTran2_EP ) *φ + phiMin
    else
        return phiMaxApp
    end
end
plot(φ_fun ,0 , uMax )

In [ ]:
uAppVec = range(0 , uMax , length = Int64(floor( tMax / delt ) ) )
AppTOption = 1 ## 1 for smooth and otherwise linear than constant
if AppTOption == 1
    TAppVec = smoothT.(uAppVec)
else
    TAppVec = Tfun.( uAppVec )
end
AppφOption = 2 ## 1 for smooth and otherwise linear than constant
if AppφOption == 1
    φAppVec = smoothφ.(uAppVec)
else
    φAppVec = φ_fun.(uAppVec)
end

In [ ]:
I2 = SymTensorValue{2 , Float64 }(1.0 ,0.0 ,1.0)
I4 = I2⊗I2
I4_sym = one(SymFourthOrderTensorValue{2,Float64})
I4_vol = (1.0/3) * I4
I4_dev = I4_sym - I4_vol

In [ ]:
const c = 421.4
const ρ = 5700.0
const Gc = 200
const η = 1e-8
const C₁₁_mat = 166e9
const C₁₂_mat = 78e9
const C₂₂_mat = 162e9
const C₆₆_mat = 43e9
const e₂₁_mat = -4.4
const e₂₂_mat = 18.6
const e₁₆_mat = 11.6
const K₁₁_mat = 11.2e-9
const K₂₂_mat = 12.6e-9
const α₁₁_mat = 15.7e-6
const α₂₂_mat = 6.4e-6
κ_mat = 3.2

In [ ]:
const L = 1e-3
const H = 1e-3
const lsp = L/90
const CP = H/2
const CL = 0.5*L 
const CH = H/2000
const hfc = lsp/5
const hf = lsp/4
const h = 20*hf 
const FMR = 6*lsp
 
gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.geo.addPoint(0.0, 0.0, 0.0, h, 1)  
gmsh.model.geo.addPoint(L, 0.0, 0.0, h, 2) 
gmsh.model.geo.addPoint(L, H, 0.0, h, 3) 
gmsh.model.geo.addPoint(0.0, H, 0.0, h, 4)
gmsh.model.geo.addPoint(0.0, CP + CH, 0.0, h, 5)
gmsh.model.geo.addPoint(CL, CP + CH, 0.0, hfc, 6)
gmsh.model.geo.addPoint(CL, CP - CH, 0.0, hfc, 7)
gmsh.model.geo.addPoint(0.0, CP - CH, 0.0, h, 8)

gmsh.model.geo.addLine(1, 2, 1)
gmsh.model.geo.addLine(2, 3, 2)
gmsh.model.geo.addLine(3, 4, 3)
gmsh.model.geo.addLine(4, 5, 4)
gmsh.model.geo.addLine(5, 6, 5)
gmsh.model.geo.addLine(6, 7, 6)
gmsh.model.geo.addLine(7, 8, 7)
gmsh.model.geo.addLine(8, 1, 8)
gmsh.model.geo.addCurveLoop([1,2,3,4,5,6,7,8],1) 
gmsh.model.geo.addPlaneSurface([1], 1)
gmsh.model.addPhysicalGroup(2, [1],1)
gmsh.model.addPhysicalGroup(1, [1],1)
gmsh.model.addPhysicalGroup(1, [3],2)
gmsh.model.setPhysicalName(2, 1, "Domain")
gmsh.model.setPhysicalName(1, 1, "BottomEdge")
gmsh.model.setPhysicalName(1, 2, "TopEdge")


gmsh.model.mesh.field.add("Box", 11)
gmsh.model.mesh.field.setNumber(11, "VIn", hf)
gmsh.model.mesh.field.setNumber(11, "VOut", h)
gmsh.model.mesh.field.setNumber(11, "XMin", 0.8*CL)
gmsh.model.mesh.field.setNumber(11, "XMax", L)
gmsh.model.mesh.field.setNumber(11, "YMin", CP-FMR)
gmsh.model.mesh.field.setNumber(11, "YMax", CP+FMR)
gmsh.model.mesh.field.setAsBackgroundMesh(11)

gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)
gmsh.write("SquarePlateWithEdgeNotch.msh")
gmsh.finalize()

In [ ]:
model = GmshDiscreteModel("SquarePlateWithEdgeNotch.msh")
writevtk(model , "SquarePlateWithEdgeNotch" )

In [ ]:
using Gridap.Geometry
labels = get_face_labeling(model)
dimension = 2
mat_tags = get_face_tag(labels,dimension)
const Mat_tag = get_tag_from_name(labels,"Domain")

In [ ]:
function ElasFourthOrderConstTensor(C11,C12,C22,C66)
    C1111 = C11
    C1122 = C12
    C1112 = 0.0
    C2222 = C22
    C2212 = 0.0
    C1212 = C66
    C_ten = SymFourthOrderTensorValue(C1111,C1112,C1122,C1112,C1212,C2212,C1122,C2212,C2222)
    return C_ten
end
const C_mat = ElasFourthOrderConstTensor(C₁₁_mat,C₁₂_mat,C₂₂_mat,C₆₆_mat)

In [ ]:
function PiezoThirdOrderConstTensor(e21,e22,e16)
    # 1 for Plane Stress and 2 Plane Strain Condition
    e111 = 0.0
    e112 = e16
    e121 = e16
    e122 = 0.0
    e211 = e21
    e212 = 0.0
    e221 = 0.0
    e222 = e22
    vals = zeros(2 ,2 ,2);
    vals[1 ,: ,:] .= [ e111 e112
                       e121 e122 ]
    vals[2 ,: ,:] .= [ e211 e212
                       e221 e222 ]
e_ten = ThirdOrderTensorValue(vals...)
return e_ten
end
const e_mat = PiezoThirdOrderConstTensor(e₂₁_mat,e₂₂_mat,e₁₆_mat)

In [ ]:
const K_mat = TensorValue(K₁₁_mat ,0.0 ,0.0 ,K₁₁_mat )

In [ ]:
const α_mat = TensorValue(α₁₁_mat ,0.0 ,0.0 , α₂₂_mat )

In [ ]:
σ_elas(ε) = C_mat ⊙ ε
function σ_elasMod(ε, ε_in , s_in , T_in , T )
    εElas_in = ε_in - α_mat *( T_in - T0 )
    εElas = ε - α_mat *( T - T0 )
    if tr(εElas_in) >= 0
        σ = (s_in ^2 + η) *σ_elas(εElas )
    else
        σ = (s_in ^2 + η) * I4_dev ⊙ σ_elas(εElas) + I4_vol ⊙ σ_elas(εElas)
    end
    return σ
end

In [ ]:
function σ_totMod1(ε,ε_in,s_in,T_in)
    εElas_in = ε_in - α_mat *( T_in - T0 )
    εElasTot1 = ε
    if tr(εElas_in) >= 0
        σT1 = (s_in ^2 + η) * σ_elas(εElasTot1)
    elseif tr(εElas_in) < 0
        σT1 = (s_in ^2 + η) * I4_dev ⊙ σ_elas(εElasTot1 ) + I4_vol ⊙ σ_elas(εElasTot1 )
    end
    return σT1
end

function σ_totMod2(ε_in,s_in,T,T_in)
    εElas_in = ε_in - α_mat *( T_in - T0 )
    εElasTot2 = - α_mat * T 
    if tr(εElas_in ) >= 0
        σT2 = (s_in^2 + η) *σ_elas(εElasTot2)
    elseif tr(εElas_in) < 0
        σT2 =  (s_in ^2 + η) * I4_dev ⊙ σ_elas(εElasTot2) + I4_vol ⊙ σ_elas(εElasTot2) 
    end
    return σT2
end

In [ ]:
function σ_thermMod(ε_in,s_in,T_in)
    εElas_in = ε_in - α_mat *( T_in - T0 )
    εElasTher = α_mat *( T0 )
    if tr(εElas_in) >= 0
        σF = (s_in ^2 + η) * σ_elas(εElasTher)
    elseif tr(εElas_in) < 0
        σF = (s_in ^2 + η) * I4_dev ⊙ σ_elas(εElasTher) + I4_vol⊙ σ_elas(εElasTher)
    end
    return σF
end

In [ ]:
σ_piezo(∇) = ∇ ⋅ e_mat

σ_piezoMod(∇, s_in) = (s_in^2 + η)*σ_piezo(∇)

function D_elasMod1(ε, ∇_in , s_in )
    return (s_in ^2 + η) * (e_mat ⋅² ε)
end

function D_elasMod2(T,s_in)
    εElas = α_mat * T 
    return (s_in ^2 +η)*(e_mat ⋅² εElas)
end

function D_thermMod( s_in )
    εElas = α_mat * T0 
    return ( s_in ^2 +η) *(e_mat ⋅² εElas)
end

D_piezo(∇) = K_mat ⋅ ∇
function D_piezoMod(∇, s_in)
    return (s_in ^2 + η)*D_piezo(∇)
end


κGradTemp(∇, s_in) = (s_in ^2 + η) *κ_mat * ∇

In [ ]:
function ψPos(ε_in , T_in , ∇_in )
    εElas_in = ε_in - α_mat *( T_in - T0 )
    if tr(εElas_in) >= 0
        ψPlus = 0.5*(ε_in ⊙ σ_elas(εElas_in)) + 0.5*(σ_piezo(∇_in ) ⊙ εElas_in )
    elseif tr(εElas_in ) < 0
        ψPlus = 0.5*(( I4_dev ⊙ σ_elas(εElas_in ) ) ⊙ ( I4_dev ⊙ ε_in ) ) +0.5*(σ_piezo(∇_in) ⊙ ( I4_dev ⊙ εElas_in ) )
    end
    return ψPlus
end

In [ ]:
function new_EnergyState(ψPlusPrev_in ,ψhPos_in )
    ψPlus_in = ψhPos_in
    if ψPlus_in >= ψPlusPrev_in
        ψPlus_out = ψPlus_in
    else
        ψPlus_out = ψPlusPrev_in
    end
    true ,ψPlus_out
end

In [ ]:
function project(q,model,dΩ,order)
    reffe = ReferenceFE(lagrangian,Float64,order)
    V = FESpace(model,reffe,conformity =:L2)
    a(u,v) = ∫( u * v )*dΩ
    l(v) = ∫( v * q )*dΩ
    op = AffineFEOperator(a,l,V,V)
    qh = Gridap.solve(op)
    qh
end

In [ ]:
order = 1
degree = 2* order
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)

In [ ]:
LoadTagId = get_tag_from_name(labels,"TopEdge")
Γ_Load = BoundaryTriangulation( model,tags = LoadTagId)
dΓ_Load = Measure(Γ_Load , degree )
n_Γ_Load = get_normal_vector(Γ_Load)

In [ ]:
reffe_PF = ReferenceFE(lagrangian,Float64,order)
V0_PF = TestFESpace(model , reffe_PF ;conformity =:H1)
U_PF = TrialFESpace(V0_PF)
sh = zero(V0_PF)

In [ ]:
reffe_Disp = ReferenceFE(lagrangian,VectorValue{2,Float64},order)
V0_Disp = TestFESpace(model , reffe_Disp ;
    conformity =:H1 ,
    dirichlet_tags =["TopEdge" ,"BottomEdge"] ,
    dirichlet_masks =[(false,true),(true,true)])
uh = zero( V0_Disp )

reffe_ElecPot = ReferenceFE(lagrangian,Float64,order)
V0_ElecPot = TestFESpace(model,reffe_ElecPot ;
    conformity =:H1 ,
    dirichlet_tags =["TopEdge","BottomEdge"])
φh = zero( V0_ElecPot )

reffe_Temp = ReferenceFE( lagrangian , Float64 , order )
V0_Temp = FESpace(model, reffe_Temp ;
    conformity =:H1 ,
    dirichlet_tags =["TopEdge","BottomEdge"])
Th = zero(V0_Temp) 

In [ ]:
V0 = MultiFieldFESpace([V0_Disp , V0_ElecPot , V0_Temp])

In [ ]:
function  stepPhaseField(uh_in,ψPlusPrev_in)
        
        a_PF(s,ϕ) = ∫( Gc*lsp*∇(ϕ)⋅ ∇(s) + 2*ψPlusPrev_in*s*ϕ  + (Gc/lsp)*s*ϕ )*dΩ
        b_PF(ϕ) = ∫( (Gc/lsp)*ϕ )*dΩ
        op_PF = AffineFEOperator(a_PF,b_PF,U_PF,V0_PF)
        sh_out = Gridap.solve(op_PF)           
    
    return sh_out
    
end

In [ ]:
function stepDispElecPotTemp(uh_in,phih_in,T_in,sh_in,vApp,TApp,phiApp,delt)
    uApp1(x) = VectorValue(0.0,vApp)
    uApp2(x) = VectorValue(0.0,0.0)
    U_Disp = TrialFESpace(V0_Disp ,[uApp1,uApp2])
    phiApp1(x) = phiApp
    phiApp2(x) = 0.0
    U_ElecPot = TrialFESpace(V0_ElecPot ,[phiApp1,phiApp2])
    Tapp1(x) = TApp
    Tapp2(x) = T0
    Tg = TrialFESpace(V0_Temp,[Tapp1,Tapp2])
    U = MultiFieldFESpace([U_Disp,U_ElecPot,Tg])
    a((u,φ,T),(v,ψ,w)) = ∫( (ε(v) ⊙ (σ_totMod1 ∘(ε(u),ε(uh_in),sh_in,T_in)
                + σ_totMod2∘(ε(uh_in),sh_in,T,T_in)))
                + (ε(v) ⊙ (σ_piezoMod∘(∇(φ),sh_in)))
                + (∇(ψ)⋅(D_piezoMod∘(∇(φ),sh_in)))
                - (∇(ψ)⋅(D_elasMod1∘(ε(u) ,∇(phih_in) , sh_in)))
                + ((∇(ψ)⋅(D_elasMod2∘(T,sh_in))))
                + ∇(w)⋅(κGradTemp∘(∇(T),sh_in))
                + ((ρ*c*T*w) / delt))*dΩ
    b((v,ψ,w)) = ∫(((ρ*c*T_in*w) / delt) - (ε(v)⊙(σ_thermMod∘(ε(uh_in),sh_in,T_in)))+ ∇(ψ)⋅(D_thermMod∘(sh_in)))*dΩ
    op = AffineFEOperator(a,b,U,V0)
    uhPhihTh = Gridap.solve(op)
    uh_out,phih_out,Th_out = uhPhihTh
    return uh_out,phih_out,Th_out
end

In [ ]:
stepDispElecPotTemp(uh,φh,Th,sh,0.01,320,1000,delt)

In [ ]:
t = 0.0
innerMax = 10
count = 0
tol = 1e-8
Load = Float64[]
Displacement = Float64[]
# push !( Load , 0.0)
# push !( Displacement , 0.0)
ψPlusPrev = CellState(0.0 , dΩ)
sPrev = CellState(1.0 , dΩ)
sh = project(sPrev , model , dΩ, order )
φPrev = CellState(0.0 , dΩ)
φh = project(φPrev , model , dΩ, order )
ThPrev = CellState(T0 , dΩ)
Th = project(ThPrev , model , dΩ, order )
while t.< tMax
    count = count .+ 1
    t = t + delt
    vApp = AppVel * t
    TApp = TAppVec[count]
    phiApp = φAppVec[count]
    print("\n Entering displacemtent step$count :",float(vApp))
    for inner = 1: innerMax
        ψhPlusPrev = project(ψPlusPrev , model , dΩ, order )
        RelErr = abs(sum(∫( Gc*lsp*∇(sh)⋅ ∇(sh) + 2*ψhPlusPrev*sh*sh  + (Gc/lsp)*sh*sh)*dΩ - ∫( (Gc/lsp)*sh)*dΩ))/abs(sum(∫( (Gc/lsp)*sh)*dΩ))
        print("\n Relative error = " , float(RelErr))
        sh = stepPhaseField(uh,ψhPlusPrev) 
        uh,φh,Th = stepDispElecPotTemp(uh,φh,Th,sh,vApp,TApp,phiApp,delt)
        ψhPos_in = ψPos∘(ε(uh),Th ,∇(φh))
        update_state!(new_EnergyState ,ψPlusPrev ,ψhPos_in)
        if RelErr < 1e-8
            break
        end
    end
    Node_Force = sum(∫( n_Γ_Load ⋅(σ_elasMod∘(ε(uh),ε(uh),sh,Th,Th))) * dΓ_Load+ ∫( n_Γ_Load ⋅(σ_piezoMod∘(∇(φh) , sh ) ) ) * dΓ_Load )
    push!( Load , Node_Force[2])
    push!( Displacement , vApp )
    if mod(count ,10) == 0
        writevtk(Ω," results_TEMNotchedPlate$count" , cellfields =
            [ "uh" => uh , "s" => sh ,"phi" => φh , "epsi" => ε(uh) , "T" => Th ])
    end
end

In [ ]:
writevtk(Ω," results_TEMNotchedPlate$count " , cellfields =
    ["uh" => uh , "s" => sh ,"phi" =>φh , "epsi" =>ε(uh) , "T" => Th ])

In [ ]:
plot(Displacement,(Load)*1e-3)